In [ ]:
(require '[clojupyter.misc.helper :as helper])
(helper/add-dependencies '[metasoarous/oz "1.5.6"])
(require '[oz.notebook.clojupyter :as oz])
(helper/add-dependencies '[clj-time "0.15.2"])
(require '[clj-time.format :as f])


(def deaths-global-csv
  "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")

(def cases-global-csv
  "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")

(defn parse-global-csv [file]
  (map (fn [line] (clojure.string/split line #","))
       (clojure.string/split-lines
        (slurp file))))


(defn parse-date [in]
  (f/parse (f/formatter "MM/dd/YY") in))

(defn get-timestamps [dataset]
  (->>
   (take 1 dataset) ;; header is first row; data is on second
   (first) ;; first vector in the list ([ .... ])
   (drop 4) ;; data starts on 5th column
   (map parse-date) ;; parse string as date
   (map #(.getMillis %)))) ;; convert to millis


(defn counts-for-country-and-provinces [country dataset]
    (apply mapv + (for [row (filter  #( if (and (= (% 1) country)) %) dataset)] 
        (map #(Integer/parseInt %) (drop 4 row))))
)

(defn counts-for-country [country dataset]
  (->> (filter  #( if (and (= (% 1) country) (= (% 0) "")) %) dataset)
     (first)
     (drop 4)
     (map #(Integer/parseInt %))))


(defn counts-per-day [dataset] 
    (map - (rest dataset) dataset))


;; http://www.learningclojure.com/2010/03/moving-average-of-list.html
(defn partial-sums [start lst]
  (lazy-seq
    (if-let [lst (seq lst)]
      (cons start (partial-sums (+ start (first lst)) (rest lst)))
      (list start))))

(defn rolling-average [window lst]
  (map #(/ % window)
       (let [start   (apply + (take window lst))
             diffseq (map - (drop window lst) lst)]
         (partial-sums start diffseq))))


(def deaths-global-data (parse-global-csv deaths-global-csv))
(def cases-global-data (parse-global-csv cases-global-csv))


(def country-population
  {
   :US 328000000
   :Russia 144400000
   (keyword "United Kingdom") 66650000
   :Germany 83000000
   :France 67006000
   :Ireland 4969000
   :Hungary 9770000
   :Greece 10720000
   :Austria 8852000
   :Italy 60360000
   :Denmark 5806000
   :Sweden 10230000
   :Norway 5328000
   :Japan 126300000
   :Finland 5518000
   :China 1398000000
   :Romania 19164000})


;; custom country list
;; make sure you add above the population size
(def countries ["Ireland" "Romania" "Hungary" "Greece" "Austria" "Denmark" "Finland" "Norway" "Sweden"])



;; commment this one if you use custom list like above
;; (def countries (map name (keys country-population)))

(defn generate-oz-values [dataset countries]
                    (let [timestamps (get-timestamps dataset)]
                      (for [country countries
                            :let [ values (counts-for-country country dataset)]]
                        (map #(hash-map :time %1 :item country :quantity %2) timestamps values))))

In [ ]:
(def cases-total
  {:title "Cases Total"
   :data {:values (flatten (generate-oz-values cases-global-data countries))}
   :mark "line"
   :height 400
   :width 800
   :encoding {:x {:field "time"  :timeUnit "yearmonthday" :type "temporal" :title "Time"}
              :y {:field "quantity" :type "quantitative" :title "Cases"}
              :color {:field "item"}}})

(oz/view! cases-total)

In [ ]:
(def cases-daily-data 
    (let [timestamps (get-timestamps cases-global-data)]
        (for [country countries 
              :let [ values  (rolling-average 7 
                                              (counts-per-day (counts-for-country-and-provinces country cases-global-data)))]]
                        (map #(hash-map :time %1 :item country :quantity %2) timestamps values))))

(def cases-daily-view
  {:title "Cases Daily - 7 days average"
   :data {:values (flatten cases-daily-data)}
   :mark "line"
   :height 400
   :width 800
   :encoding {:x {:field "time"  :timeUnit "yearmonthday" :type "temporal" :title "Time"}
              :y {:field "quantity" :type "quantitative" :title "Cases"}
              :color {:field "item"}}})

(oz/view! cases-daily-view)

In [ ]:
(def cases-7days-average-data
    (flatten (let [timestamps (get-timestamps cases-global-data)] 
                 (for [country countries 
                       :let [ values (rolling-average 7 
                                        (counts-per-day 
                                            (counts-for-country-and-provinces country cases-global-data)))
                              population (country-population (keyword country))]]
                        (map #(hash-map 
                        :time %1 
                        :item country 
                        :quantity (* (/ %2 population) 100000)) timestamps values)))))

(def cases-7days-view
  {:title "Cases Daily per 100k population- 7 days average"
   :data {:values (flatten cases-7days-average-data)}
   :mark "line"
   :height 400
   :width 800
   :encoding {:x {:field "time"  :timeUnit "yearmonthday" :type "temporal" :title "Time"}
              :y {:field "quantity" :type "quantitative" :title "Cases"}
              :color {:field "item"}}})

(oz/view! cases-7days-view)

In [ ]:
(def deaths-total
  {:title "Deaths Total"
   :data {:values (flatten (generate-oz-values deaths-global-data countries))}
   :mark "line"
   :height 400
   :width 800
   :encoding {:x {:field "time"  :timeUnit "yearmonthday" :type "temporal" :title "Time"}
              :y {:field "quantity" :type "quantitative" :title "Deaths"}
              :color {:field "item"}}})

              
(oz/view! deaths-total)

In [ ]:
(def deaths-per-7days-data
    (flatten (let [timestamps (get-timestamps deaths-global-data)] 
                 (for [country countries 
                       :let [ values (rolling-average 7 (counts-per-day 
                                        (counts-for-country-and-provinces country deaths-global-data)))]]
                        (map #(hash-map :time %1 :item country :quantity %2) timestamps values)))))





(def deaths-per-7days-view
  {:title "Deaths/day - 7 days rolling average average"
   :data {:values deaths-per-7days-data}
   :mark "line"
   :height 400
   :width 800
   :encoding {:x {:field "time"  :timeUnit "yearmonthday" :type "temporal" :title "Time"}
              :y {:field "quantity" :type "quantitative" :title "Deaths"}
              :color {:field "item"}}})

              
(oz/view! deaths-per-7days-view)

In [ ]:
(def deaths-100k-per-7days-data
    (flatten (let [timestamps (get-timestamps deaths-global-data)] 
                 (for [country countries 
                       :let [ values (rolling-average 7 
                                        (counts-per-day 
                                            (counts-for-country-and-provinces country deaths-global-data)))
                              population (country-population (keyword country))]]
                        (map #(hash-map 
                        :time %1 
                        :item country 
                        :quantity (* (/ %2 population) 100000)) timestamps values)))))

(def deaths-100k-per-7days-view
  {:title "Deaths/day per 100k population  - 7 days rolling average"
   :data {:values deaths-100k-per-7days-data}
   :mark "line"
   :height 400
   :width 800
   :encoding {:x {:field "time"  :timeUnit "yearmonthday" :type "temporal" :title "Time"}
              :y {:field "quantity" :type "quantitative" :title "Deaths"}
              :color {:field "item"}}})
          
(oz/view! deaths-100k-per-7days-view)